# The Battle of Neighborhoods - The Best Place for Cafe in Toronto

## Introduction

A contractor is looking to open a Cafe in Toronto, one of the most densely populated areas in Canada. He has contacted me for advice on the suitable neighborhoods to open the Cafe. In this case, we will utilize data science skills and the tools to use location data to explore Toronto neighborhoods, in order to help my contractor, and any other business owners who might be interested in starting their business in Toronto.

## Data Section

In this project, we will need the following data to solve this problem and execute our idea:
  
1. Wikipedia page of Toronto neighborhoods  
The following Wikipedia page has all the information we need to explore and cluster the neighborhoods in Toronto. We will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe of the postal code of each neighborhood along with the borough name and neighborhood name.  
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  
  
2.  Geospatial Coordinates csv file  
We will use the following link to the Geospatial_Coordinates csv file to get the latitude and the longitude coordinates of each neighborhood.   
http://cocl.us/Geospatial_data  
  
3. Foursquare API  
We will need to use the Foursquare location data, and download all venues from all neighborhoods in Toronto.  
https://api.foursquare.com


## Methodology Section

1. K-Means Clustering  
In this project, we will use K-Means Clustering algorithm to cluster the neighborhoods.

2. One Hot Encoding  
In this project, we will do one hot encoding for getting dummies of the venue category.

### Import libraries

In [2]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import json
from pandas.io.json import json_normalize

### Scrap data from Wikipedia page

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')
table = soup.find('table')

### Only process the cells that have an assigned borough. 

In [4]:
res = []
for tr in table.tbody.find_all("tr"):
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    if row != [] and row[1] != "Not assigned\n":
        if "Not assigned\n" in row[2]: 
            row[2] = row[1]
        res.append(row)

### Create a new DataFrame

In [5]:
df_1 = pd.DataFrame(res, columns = ["Postal Code", "Borough", "Neighborhood"])
df_1.head()

,Postal Code,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


### Remove "\n" 

In [6]:
df_1["Postal Code"] = df_1["Postal Code"].str.replace("\n","")
df_1["Neighborhood"] = df_1["Neighborhood"].str.replace("\n","")
df_1["Borough"] = df_1["Borough"].str.replace("\n","")
df_1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Print the number of rows of dataframe

In [7]:
df_1.shape

(103, 3)

### Load the csv file that has the geographical coordinates of each postal code.

In [8]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_18cfca85fe754779bb9b0ce40694a871 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='U5nFzJqKalgrevEQBJ1FC_AHJHYYNzbWzU793XrZ7GX9',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_18cfca85fe754779bb9b0ce40694a871.get_object(Bucket='week3projectpart1-donotdelete-pr-bmgwxfq3i8vmtu',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


###  Merge the dataframes based on Postal Code

In [9]:
df_2 = df_1.join(df_data_1.set_index('Postal Code'), on ='Postal Code')
df_2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Import libraries to get latitude and longitude valudes of Toronto

In [10]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
!pip install folium

     |████████████████████████████████| 94 kB 5.5 MB/s  eta 0:00:01


### Get geographical coordinates of Toronto

In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geographical coordinates of Toronto are {}, {}'.format(latitude_toronto, longitude_toronto))

The geographical coordinates of Toronto are 43.6534817, -79.3839347


### Use folium to produce a map for Toronto

In [12]:
import folium
map = folium.Map(location = [latitude_toronto, longitude_toronto], zoom_start = 10)

for lat, lng, borough, neighborhood in zip(df_2['Latitude'], df_2['Longitude'], df_2['Borough'], df_2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
map

### Create a dataframe to include the neighborhoods in downtown Toronto.

In [13]:
df_3 = df_1.merge(df_data_1, how='left', on='Postal Code')
df_3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
downtown_toronto = df_3[df_3['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
downtown_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [15]:
downtown_toronto.shape

(19, 5)

### Get geographical coordinates of Toronto

In [16]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="Downtown Toronto")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print('The geographical coordinates of Downtown Toronto are {}, {}'.format(latitude_downtown_toronto, longitude_downtown_toronto))

The geographical coordinates of Downtown Toronto are 43.6563221, -79.3809161


### Use folium to produce a map for Downtown Toronto

In [17]:
downtown_toronto_map= folium.Map(location = [latitude_downtown_toronto, longitude_downtown_toronto], zoom_start = 13)

for lat, lng, label in zip(df_3['Latitude'], df_3['Longitude'], df_3['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(downtown_toronto_map)

downtown_toronto_map

### Access Foresquare account info.

In [18]:
CLIENT_ID = '1MALAGQI3ZRPDREJVP54FHOIEUFUFX2OD4K4CQBZK03LF0G2' 
CLIENT_SECRET = '4TM4JAB2YVLNG2N3TA0FTBY4E3PKTHLIF3LKASVX5FL3MEQ5'
VERSION ='20180604'
LIMIT = 100

### Get latitude and longitude values for the neighborhoods.

In [19]:
neighborhood_latitude = downtown_toronto.loc[0, 'Latitude']
neighborhood_longitude = downtown_toronto.loc[0, 'Longitude']
neighborhood_name = downtown_toronto.loc[0, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359


### Define the URL and a query to search for venues that are within 500 metres.

In [20]:
search_query = ''
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,neighborhood_latitude,neighborhood_longitude,VERSION,search_query,radius,LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '603888c46db72d086ac43034'},
 'response': {'venues': [{'id': '5bdc6c2bba57b4002c4c71a8',
    'name': 'Oldtown Bodega',
    'location': {'lat': 43.653966,
     'lng': -79.360752,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.653966,
       'lng': -79.360752}],
     'distance': 34,
     'postalCode': 'M5A 1L6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto ON M5A 1L6', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1614317764',
    'hasPerk': False},
   {'id': '4bc70f5d14d7952126a066e9',
    'name': 'Sackville Playground',
    'location': {'address': '420 king st E',
     'lat': 43.65465604258614,
     'lng': -79.359870

### Get relevant part of JSON and transform it into a pandas dataframe

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

#tranform venues into a datafram
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,venuePage.id
0,5bdc6c2bba57b4002c4c71a8,Oldtown Bodega,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1614317764,False,43.653966,-79.360752,"[{'label': 'display', 'lat': 43.653966, 'lng':...",34,M5A 1L6,CA,Toronto,ON,Canada,"[Toronto ON M5A 1L6, Canada]",NaN,NaN,NaN
1,4bc70f5d14d7952126a066e9,Sackville Playground,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1614317764,False,43.654656,-79.359871,"[{'label': 'display', 'lat': 43.65465604258614...",75,NaN,CA,Toronto,ON,Canada,"[420 king st E, Toronto ON, Canada]",420 king st E,NaN,NaN
2,53b8466a498e83df908c3f21,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1614317764,False,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",122,NaN,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...",368 King St E,at Trinity St,NaN
3,55e8cc7a498e795a53d81d36,TTC Streetcar #503 Kingston Rd,"[{'id': '4f2a23984b9023bd5841ed2c', 'name': 'M...",v-1614317764,False,43.663514,-79.337697,"[{'label': 'display', 'lat': 43.66351445487503...",2115,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,NaN
4,4b0d4672f964a520854523e3,TTC Streetcar #504 King St,"[{'id': '4f2a23984b9023bd5841ed2c', 'name': 'M...",v-1614317764,False,43.677508,-79.358114,"[{'label': 'display', 'lat': 43.67750829686272...",2595,NaN,CA,Toronto,ON,Canada,"[King St. (Moving Target!), Toronto ON, Canada]",King St.,Moving Target!,NaN


### Define information of interests and filter dataframe

In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories','location.lat','location.lng']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng
0,Oldtown Bodega,Café,43.653966,-79.360752
1,Sackville Playground,Park,43.654656,-79.359871
2,Tandem Coffee,Coffee Shop,43.653559,-79.361809
3,TTC Streetcar #503 Kingston Rd,Moving Target,43.663514,-79.337697
4,TTC Streetcar #504 King St,Moving Target,43.677508,-79.358114
...,...,...,...,...
95,Take It Easy Personal Concierge,Office,43.655022,-79.359968
96,Happen Hub,Office,43.652932,-79.360675
97,Ward 28 Voting Station,City Hall,43.654830,-79.357985
98,Huayu Kitchen,Chinese Restaurant,43.654148,-79.357826


### Get the venues that are nearby

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items'] 
         # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
downtown_toronto_venues = getNearbyVenues(names = downtown_toronto['Neighborhood'],
                                    latitudes = downtown_toronto['Latitude'],
                                   longitudes = downtown_toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [27]:
downtown_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [28]:
print(downtown_toronto_venues.shape)

(1215, 7)


### Check unique categories.

In [29]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 209 uniques categories.


### Analyzing Neighborhoods

In [30]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix = '', prefix_sep='')
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood']
tmp = downtown_toronto_onehot.pop('Neighborhood')
downtown_toronto_onehot.insert(0, 'Neighborhood', tmp)

downtown_toronto_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group by Neighborhood by taking the mean of each category.

In [31]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.017241,0.000000,0.000000,0.0,0.0,0.017241,0.0,0.0,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.076923,0.076923,0.076923,0.076923,0.230769,0.076923,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.032787,0.000000,0.000000,0.0,0.0,0.016393,0.0,0.0,0.016393,0.016393
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Church and Wellesley,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.0,...,0.012658,0.012658,0.012658,0.0,0.0,0.000000,0.0,0.0,0.000000,0.025316


### Get the top 5 common venues

In [59]:
num_top_venues = 10

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Berczy Park----
                     venue  freq
0              Coffee Shop  0.10
1                   Bakery  0.05
2             Cocktail Bar  0.05
3           Farmers Market  0.03
4       Seafood Restaurant  0.03
5              Cheese Shop  0.03
6               Restaurant  0.03
7                 Beer Bar  0.03
8                 Pharmacy  0.03
9  Comfort Food Restaurant  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.23
1       Boat or Ferry  0.08
2    Sculpture Garden  0.08
3  Airport Food Court  0.08
4        Airport Gate  0.08
5      Airport Lounge  0.08
6    Airport Terminal  0.08
7     Harbor / Marina  0.08
8            Boutique  0.08
9             Airport  0.08


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1   Italian Restaurant  0.05
2       Sandwich Place  0.05
3                 Café  0.05
4      Bubble Tea 

### Represent data in pandas dataframe

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
um_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Liquor Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Sculpture Garden,Boat or Ferry,Boutique
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant
3,Christie,Grocery Store,Café,Park,Nightclub,Candy Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Coffee Shop
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Pub,Hotel,Fast Food Restaurant


### Cluster Neighborhoods

In [62]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5
downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 4, 0, 0, 0, 0, 0, 0], dtype=int32)

In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [64]:
downtown_toronto_merged = downtown_toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
downtown_toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Theater,Café,Pub,Breakfast Spot,Restaurant,Dessert Shop,Bank
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Smoothie Shop,Italian Restaurant,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Diner,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,American Restaurant,Cocktail Bar,Gastropub,Gym,Italian Restaurant,Department Store,Clothing Store,Hotel
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Liquor Store


### Use folium to create map

In [65]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start = 10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster -1],
        fill = True,
        fill_color = rainbow[cluster -1],
        fill_opacity = 0.7).add_to(map_clusters)

map_clusters

## Result Section

Below, we will have 3 clusters 0, 1 and 2 repectively. For each cluster, we will list the 5 most common venue for each Neighborhood.

### Examine Clusters

In [66]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Bakery,Park,Theater,Café,Pub,Breakfast Spot,Restaurant,Dessert Shop,Bank
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Smoothie Shop,Italian Restaurant,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center
2,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Diner,Lingerie Store
3,St. James Town,0,Café,Coffee Shop,American Restaurant,Cocktail Bar,Gastropub,Gym,Italian Restaurant,Department Store,Clothing Store,Hotel
4,Berczy Park,0,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Liquor Store
5,Central Bay Street,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant
7,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Bakery,Clothing Store,Thai Restaurant,Deli / Bodega,Gym,Burrito Place,Asian Restaurant
8,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Brewery,Scenic Lookout,Sporting Goods Shop,Fried Chicken Joint,Restaurant
9,"Toronto Dominion Centre, Design Exchange",0,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,American Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,Salad Place
10,"Commerce Court, Victoria Hotel",0,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,American Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant


In [67]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,1,Park,Trail,Playground,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [68]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[2] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",2,Airport Service,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Sculpture Garden,Boat or Ferry,Boutique


## Discussion Section

According to the analysis, Rosedale in Cluster 1 will be the best place for my contractor to open his Cafe. As we built our K-Means dataset and list of neighborhoods with their most common venues, we discovered Cluster 1 provides comparatively less competition than Cluster 0, and closer distance to residentials than Cluster 2. 

## Conclusion Section

To conclude this project, we used Python libraries to handle JSON file, plotting graphs, and other exploratory data analysis. in order to help my contractor, and any other business owners who might be interested in starting their business in Toronto to find the best place for an upcoming Cafe.